In [1]:


import numpy as np
import pandas as pd
import yfinance as yf
import plotly.graph_objs as go
import plotly.offline as py
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2
import pyfolio as pf
import plotly.express as px
from termcolor import colored as cl
import math
from plotly.offline import plot, iplot, init_notebook_mode
from collections import defaultdict
from datetime import datetime
import talib
import warnings
warnings.filterwarnings("ignore")

C:\Users\Anshu\Anaconda3\lib\site-packages\pyfolio\pos.py:26: UserWarning:

Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.



ModuleNotFoundError: No module named 'talib'

In [ ]:
df=pd.read_csv('token-data.csv')

In [ ]:
df['coin'].unique()

In [ ]:
from datetime import datetime
df['date']=pd.to_datetime(df['date'])
df=df[df['date']>datetime(2020,1,1,0,0)]
df


# **Preprocessing**

In [ ]:
dct={
    'coin':[],
    'tot':[],
    'val':[]
}


In [ ]:
dfm=df.copy()
for coin in dfm['coin'].unique():
    df=dfm[dfm['coin']==coin]
    df['mv']=df['price'].rolling(5).mean()
    df['sig']=np.sign(df['price']-df['mv'])
    df['period_ret']=(df['sig'].shift(1)*(df['price']-df['price'].shift(1)))/df['price'].shift(1)
    df=df.dropna()
    tot=df['period_ret'].cumsum().iloc[-1]
    val=df[df.index%5<2]['period_ret'].cumsum().iloc[-1]
    dct['coin'].append(coin)
    dct['tot'].append(tot)
    dct['val'].append(val)

In [ ]:
df_gg=pd.DataFrame(dct)

In [ ]:
df_gg[df_gg['tot']>0]

In [ ]:
df_gg[df_gg['tot']==df_gg['tot'].max()]

In [ ]:
df['period_ret'].cumsum().plot()

In [ ]:
df[df.index%5==0]['period_ret'].cumsum().plot(label='0')
df[df.index%5==1]['period_ret'].cumsum().plot(label='1')
df[df.index%5==2]['period_ret'].cumsum().plot(label='2')
df[df.index%5==3]['period_ret'].cumsum().plot(label='3')
df[df.index%5==3]['period_ret'].cumsum().plot(label='4')
plt.legend()

In [ ]:
df2=df.groupby('date')

In [ ]:
li_date=sorted(list(set(df['date'])))
print(len(li_date))
#Finding total dates in the data

In [ ]:
df.groupby(['coin'])['date'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False)
#Finding order of tokens by their count in dataset

In [ ]:
df3=df.groupby(['coin','date']).agg('mean')
#replacing multiple prices of a token on a given date by their mean.

In [ ]:
for index, row in df.iterrows():
    df.loc[index,'price']=df3['price'][row['coin']][row['date']]

In [ ]:
df.drop_duplicates(inplace=True)
#Since we need only one price for a given day of a token we drop the remaining ones

In [ ]:
df.groupby(['coin'])['date'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False).head(100)
#Taking first 100 tokens by their count in dataset.

In [ ]:
li_tokens=list(df.groupby(['coin'])['date'].count().reset_index(
  name='Count').sort_values(['Count'], ascending=False)['coin'].head(100))
val_tokens=['tether','usd-coin','binancecoin','matic-network','okb','theta-token','wrapped-bitcoin','quant-network','crypto-com-chain','huobi-token','true-usd','kucoin-shares','chiliz','gemini-dollar','compound-usd-coin','maker','rocket-pool','nexo','zilliqa','enjincoin','loopring','gnosis','celsius-degree-token','swissborg','band-protocol','golem','iotex','telcoin','coinex-token','livepeer','one']
req_tokens=['tether','usd-coin','binancecoin','binance-usd','matic-network','staked-ether','okb','theta-token','dai','shiba-inu-wormhole','hex-ethw','uniswap-wormhole','wrapped-bitcoin','chainlink-wormhole','quant-network','vendetta-finance','near','frax','huobi-token','usdp','lido-dao-wormhole','true-usd','usdd','kucoin-shares','chiliz','bittorrent','gemini-dollar','maker', 'paxos-gold-wormhole','tether-gold']
#li_tokens is list of top 100 tokens by count in dataset
#val_tokens is list of top30 token by market cap
#req_tokens is list of top15 token by market cap

# Moving Average Calculation

In [ ]:
def ema(values, period):
    df=pd.DataFrame(values)
    df['ema']=df[0].ewm(span=period).mean()
    return list(df['ema'])

def hma(values, period):
    df=pd.DataFrame(values)
    df['ema']=df[0].ewm(span=period).mean()
    s=df[0]
    HMA = s.rolling(period//2).apply(lambda x: ((np.arange(period//2) + 1)*x).sum()/(np.arange(period//2) + 1).sum(), raw=True).multiply(2).sub(
                            s.rolling(period).apply(lambda x: ((np.arange(period) + 1)*x).sum()/(np.arange(period) + 1).sum(), raw=True)
                    ).rolling(int(np.sqrt(period))).apply(lambda x: ((np.arange(int(np.sqrt(period))) + 1)*x).sum()/(np.arange(int(np.sqrt(period))) + 1).sum(), raw=True)
    return list(df['ema'])[:period]+list(HMA)[period:]

ma_30=defaultdict(lambda:0)
ema_30=defaultdict(lambda:0)
hma_30=defaultdict(lambda:0)
filled_price=defaultdict(lambda:0)
ret=defaultdict(lambda:0)

for token in li_tokens:
    sum=0
    tot=0
    taken=[1]
    for date in li_date:
        try:
            taken.append(df3['price'][token][date]+1)
            indx=len(taken)-2
            while(indx>=0 and taken[indx]==0):
                taken[index]=taken[-1]
                indx+=-1
        except:
            taken.append(taken[-1])
    taken.pop(0)
    #print(taken.count(0))
    indx=0
    for date in li_date:
        filled_price[(token,date)]=taken[indx]
        indx+=1
    em=ema(taken,23)
    hm=hma(taken,25)
    indx=0
    for date in li_date:
        sum+=taken[indx]
        ema_30[(token,date)]=em[indx]
        hma_30[(token,date)]=hm[indx]
        if(indx<30):
            ma_30[(token,date)]=sum/(indx+1)
        else:
            sum-=taken[indx-30]
            ma_30[(token,date)]=sum/30
        
        if(indx>0):
            ret[(token,date)]= np.log(taken[indx]/taken[indx-1])
        else:
            ret[(token,date)]= 0
            
        indx+=1   

**Plots**

In [ ]:
def plot1(df,buy_price,sell_price,c_price):
    plt.figure(figsize=(25,10))
    ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)



    ax1.plot(df['price'], color = 'skyblue', linewidth = 2, label = 'CLOSE')
    ax1.plot(df.index, buy_price, marker = '^', color = 'green', markersize = 7, label = 'HOLD SIGNAL', linewidth = 0)
    ax1.plot(df.index, sell_price, marker = 'v', color = 'red', markersize = 7, label = 'SELL SIGNAL', linewidth = 0)
    ax1.plot(df.index, c_price, marker = '<', color = 'blue', markersize = 7, label = 'LEAVE SIGNAL', linewidth = 0)
#     plt.savefig('price.png')
    ax1.legend()



def plot2(df):
    fig = make_subplots(rows=2, cols=1)

    fig.append_trace(go.Scatter(
       x=df.index,
       y=df['price'],
     name="price"), row=1, col=1)

    fig.append_trace(go.Scatter(
       x=df.index,
       y=df['tot'],name='total tokens'
    ), row=2, col=1)


    fig.update_layout(height=500, width=1000, title_text="Subplots")
    fig.show()
    
def plot3(df,buy_price,sell_price):
#     plt.figure(figsize=(25,10))
#     ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)


#     ax1.plot(df['tot'], color = 'skyblue', linewidth = 2, label = 'CLOSE')
#     ax1.plot(df.index, buy_price, marker = '^', color = 'green', markersize = 10, label = 'BUY SIGNAL', linewidth = 0)
#     ax1.plot(df.index, sell_price, marker = 'v', color = 'r', markersize = 10, label = 'SELL SIGNAL', linewidth = 0)
#     ax1.legend()
    fig = px.line(df, x=df.index, y=df['tot'])

    # Add Scatter plot
    indx=0
    for date in df.index:
        if(sell_price[indx]!=np.nan):
            fig.add_trace(go.Scatter(x=[date], y=[sell_price[indx]],showlegend=False, mode = 'markers',
                                    marker=dict(
                symbol='arrow-down',
                color="red",
                size=16
            )))
        if(buy_price[indx]!=np.nan):
            fig.add_trace(go.Scatter(x=[date], y=[buy_price[indx]],showlegend=False, mode = 'markers',
                                    marker=dict(
                symbol='arrow-up',
                color="green",
                size=16
            )))
        indx+=1

    # Display the plot
    fig.show()
    

    # Display the plot
    fig.show()
    
def plot4(df,tc,buy_price,sell_price,c_price):
    fig = make_subplots(rows=2, cols=1)

    fig.append_trace(go.Scatter(
       x=df.index,
       y=df['price'],
     name="price"), row=1, col=1)
    indx=0
    for date in df.index:
        if(sell_price[indx]!=np.nan):
            fig.append_trace(go.Scatter(x=[date], y=[sell_price[indx]],showlegend=False, mode = 'markers',
                                    marker=dict(
                symbol='arrow-down',
                color="red",
                size=10
            )),row=1,col=1)
        if(buy_price[indx]!=np.nan):
            fig.append_trace(go.Scatter(x=[date], y=[buy_price[indx]],showlegend=False, mode = 'markers',
                                    marker=dict(
                symbol='arrow-up',
                color="green",
                size=10
            )),row=1,col=1)
        if(c_price[indx]!=np.nan):
            fig.append_trace(go.Scatter(x=[date], y=[c_price[indx]],showlegend=False, mode = 'markers',
                                    marker=dict(
                symbol='arrow-right',
                color="black",
                size=10
            )),row=1,col=1)
        
        indx+=1

    fig.append_trace(go.Scatter(
       x=df.index,
       y=df['tot'],name='total tokens',line_color='orange'
    ), row=2, col=1)


    fig.update_layout(height=500, width=1000, title_text=tc)
    fig.show()



def cci_30(tokens,date,sign):
    sm=0
    for token in tokens:
        sm+=ema_30[(token,date)]
    re=0
    for token in tokens:
        w=math.sqrt(ema_30[(token,date)])/math.sqrt(sm)
        re+=w*sign[token]*ret[(token,date)]
    return re

def plot_returns(df):
    fig = px.line(df, x=df.index, y=df['cci_returns'])
    fig.show()


            
  




**Strategy**

In [ ]:
def trade1(data,tc,c1=49,c2=10,c3=35):
#     df=data.copy()
#     df['bnh_returns']=[ret[(row['coin'],row['date'])] for index,row in df.iterrows()]

#     df['ma30'] = [ma_30[(row['coin'],row['date'])] for index,row in df.iterrows()]
#     #df['std'] = df['Adj Close'].rolling(window=20).std()
#     df['price']=  [filled_price[(row['coin'],row['date'])] for index,row in df.iterrows()]
# c1 is number of days you go for short signal once the call is generated.
# c2 is the number of days you wait for shorting once the call is generated.
# c3 is the number of days you go for long signal once the call is generated.
    
    str_returns=defaultdict(lambda:0)
    cci_returns=defaultdict(lambda:0)
    sign=defaultdict(lambda:0)
    sm=defaultdict(lambda:0)
    last=0
    #signal=defaultdict(lambda:0)
    indx=1
    format = '%Y-%m-%d'
    li=[]
    buy_price = []
    sell_price = []
    c_price=[]
    buy_price1 = []
    sell_price1 = []
    c_price1=[]
    last1=0
    last2=0
    #c1=[750:930]
    #c2=[1200:1530]
    dates=li_date[1200:]
    last_tot=0
    for date in dates:
        buy_price.append(np.nan)
        sell_price.append(np.nan)
        c_price.append(np.nan)
        buy_price1.append(np.nan)
        sell_price1.append(np.nan)
        c_price1.append(np.nan)
        z=[]
        for token in li_tokens:
            z.append([filled_price[(token,date)],token])
        z.sort(reverse=True)
        res=[i[1] for i in z[:31]]
     
        tot=0
        for token in res:
            if(filled_price[(token,date)]>ema_30[(token,date)]):
                tot+=1
        li.append(tot)
        #print(date,tot-last_tot)
        k=tot-last_tot
        for token in li_tokens:
            if(token!=tc):
                continue
#             print(date,sign[token],ret[(token,li_date[indx])])
            str_returns[datetime.strptime(date, format)]+=sign[token]*ret[(token,date)]
            sm[datetime.strptime(date, format)]+=ret[(token,date)]
        cci_returns[datetime.strptime(date, format)]=cci_30(val_tokens,date,sign)
        
        #cooldown period of short call is over
        if(last==1):
            last=0
            if(sign[tc]==0):
                pass
            else:
                c_price[-1]=filled_price[(tc,date)]
                c_price1[-1]=tot


                for token in li_tokens:
                    sign[token]=0
            last=0
        #cooldown period of long call is over.
        if(last2>c3):
            #print(date,last2)
            last2=0
            if(sign[tc]==0):
                pass
            else:
                c_price[-1]=filled_price[(tc,date)]
                c_price1[-1]=tot


                for token in li_tokens:
                    sign[token]=0
        
        #waiting period before actually shorting is over and you can short now.
        if(last1==1):
            #print(date,last1,last2)
            if(last2>=1 and last2<=c3):
                
                
                last1+=c3-last2+1
                last+=c3-last2+1
            else:
           
                sell_price[-1]=filled_price[(tc,date)]


                for token in li_tokens:
                    if(sign[token]==0):
                        sign[token]=-1
        
        #condition triggering call for long
        if(tot<4 and last==0 and last2==0):
            buy_price[-1]=filled_price[(tc,date)]
            buy_price1[-1]=tot
            for token in li_tokens:
                sign[token]=1
            last2=1
            
            
            
        #condition triggering call for short
        if(tot>24 and last==0):
            if(tot>31):
                pass
            else:
                sell_price1[-1]=tot
                last=c1
                last1=c2
        if(last>0):
            last-=1
        if(last2>0):
            last2+=1
        if(last1>0):
            last1-=1
        last_tot=tot
            
        
            
        
#         if(tot<3):
#             for token in li_tokens:
#                 if(sign[token]==-1):
#                     sign[token]=0
        indx+=1
    df_returns=pd.DataFrame(str_returns,index=[0]).T
    df_returns.rename(columns={0:'strategy_returns'},inplace=True)
    df_returns['bnh_returns']=[sm[date] for date in df_returns.index]
    df_returns['cci_returns']=[cci_returns[date] for date in df_returns.index]
    df_returns['bnh_returns']=[sm[date] for date in df_returns.index]
    df_returns['strategy_returns'].cumsum()[-1]
    #return df_returns['strategy_returns'].cumsum()[-1]
    print("Buy and hold returns:",df_returns['bnh_returns'].cumsum()[-1])
    print("Strategy returns:",df_returns['strategy_returns'].cumsum()[-1])
    df_returns[['cci_returns']] = df_returns[['cci_returns']].cumsum()

    df_returns['price']=[filled_price[(tc,dates[i])] for i in range(len(df_returns))]
    plot1(df_returns,buy_price,sell_price,c_price)

#     plt.figure(figsize=(20, 8))
#     plt.subplot(1,2,1)
#     plt.plot(df_returns['price'])
    df_returns['tot']=[li[i] for i in range(len(df_returns))]
#     plt.subplot(1,2,2)
#     plt.plot(df_returns.index,li)
    plot2(df_returns)
    plot_returns(df_returns)
    plot3(df_returns,buy_price1,sell_price1)
    plot4(df_returns,tc,buy_price,sell_price,c_price)
#     fig=go.Figure([go.Scatter(x=df_returns.index, y=df_returns['tot'])])
#     fig.show()
    #plotting strategy historical performance over time
 
    df_returns[['bnh_returns','strategy_returns']] = df_returns[['bnh_returns','strategy_returns']].cumsum()
    df_returns[['bnh_returns','strategy_returns']].plot(grid=True, figsize=(12, 8))
  
    
    pf.create_simple_tear_sheet(df_returns['strategy_returns'].diff())







        
        
    

In [ ]:
tc='wrapped-bitcoin'
trade1(df,tc,47,7,33)

In [2]:
import sys
print(sys.version)

3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
